### Dining Philosophers with Monitors (Java)

Implement the Dining Philosophers problem with monitors in Java! Consider two designs. In both, designs, philosopher `ph` picks up first forks `ph` and then `(ph + 1) mod 5`:

1. Every fork is a monitor object with two methods, `pickup()` and `putdown()`. Forks are then binary semaphores.
2. All forks are managed by a single monitor object of class `Table`, with two methods, `pickup(int ph)`, which picks up both forks, and `putdown(int f)`, which puts down both forks.

Use `synchronized (System.out) {System.out.println(...);}` to print when each philosopher is thinking and eating. You may slow down the philosophers with `Thread.sleep(millisec);` while they are thinking and eating. Can each of the designs deadlock? If so, provide a solution.

*Note:* If you have the philosophers run in an infinite loop, you may still forcefully terminate them. For this, make each philosopher a daemon thread by `setDaemon(true)` and have the main method wait with `Thread.sleep(millisec)` before terminating. Then termination of the main thread will cause all daemon threads to be terminated as well.

In [ ]:
%%writefile philosophers.java
class Fork {
    // YOUR CODE HERE
}
class Philosopher extends Thread {
    // YOUR CODE HERE
}
class DiningPhilosophers {
    public static void main(String[] args) {
        Fork[] fork = new Fork[5];
        for (int i = 0; i < 5; i++) fork[i] = new Fork();
        // YOUR CODE HERE
    }
}

In [ ]:
%%writefile philosophers.java
class Fork {
    boolean busy = false;
    synchronized void pickup() throws InterruptedException {
        if (busy) wait(); // every fork is used only by two threads, one calling pickup and
        // the other putdown, so spurious wakeups are not possible and "if" is sufficient
        busy = true;
    }
    synchronized void putdown() {
        busy = false;
        notify();
    }
}
class Philosopher extends Thread {
    int i;
    Fork first, second;
    Philosopher(int i, Fork first, Fork second) {
        this.i = i; this.first = first; this.second = second; setDaemon(true);
    }
    public void run() {
        while (true) {
            try { // locking System.out for printing produces better output
                synchronized (System.out) {System.out.println(i + " is thinking");}
                Thread.sleep(100); // milliseconds
                first.pickup(); second.pickup();
                synchronized (System.out) {System.out.println(i + " is eating");}
                Thread.sleep(100); // milliseconds
                first.putdown(); second.putdown();  
            } catch (Exception e) {System.out.println("oops");}
        }
    }
}
class DiningPhilosophers {
    public static void main(String[] args) {
        Fork[] fork = new Fork[5];
        for (int i = 0; i < 5; i++) fork[i] = new Fork();
        /* Follwing line CAN lead to deadlock; restart the kernel after a deadlock */
        // for (int i = 0; i < 5; i++) new Philosopher(i, fork[i], fork[(i + 1) % 5]).start();        
        /* A fix is to have every philsopher pick up the lower-numbered fork first */
        for (int i = 0; i < 4; i++) new Philosopher(i, fork[i], fork[i + 1]).start();
        Philosopher(4, fork[0], fork[4]).start();
        try {Thread.sleep(3000); // milliseconds to sleep before program stops
        } catch (Exception e) {}
    }
}

In [ ]:
!javac philosophers.java

In [ ]:
!java DiningPhilosophers

In [ ]:
%%writefile philosophers.java
class Table {
    YOUR CODE HERE
    synchronized void pickup(int ph) throws InterruptedException {
        YOUR CODE HERE
    }
    synchronized void putdown(int ph) {
        YOUR CODE HERE
    }
}
class Philosopher extends Thread {
    YOUR CODE HERE
}
class DiningPhilosophers {
    public static void main(String[] args) {
        Table table = new Table();
        for (int i = 0; i < 5; i++) new Philosopher(table, i).start();
        try {Thread.sleep(3000); // milliseconds to sleep before program stops
        } catch (Exception e) {}
    }
}

In [ ]:
%%writefile philosophers.java
class Table {
    boolean[] fork = new boolean[5]; // initially false
    synchronized void pickup(int ph) throws InterruptedException {
        // a philosopher who calls pickup will either get both forks or no fork
        while (fork[ph] || fork[(ph + 1) % 5]) wait();
        fork[ph] = true; fork[(ph + 1) % 5] = true;
    }
    synchronized void putdown(int ph) {
        fork[ph] = false; fork[(ph + 1) % 5] = false;
        notifyAll();
    }
}
class Philosopher extends Thread {
    Table t;
    int ph;
    Philosopher(Table t, int ph) {
        this.t = t; this.ph = ph; setDaemon(true);   
    }
    public void run() {
        while (true) {
            try { // locking System.out for printing produces better output
                synchronized (System.out) {System.out.println(ph + " is thinking");}
                Thread.sleep(100); // milliseconds
                t.pickup(ph);
                synchronized (System.out) {System.out.println(ph + " is eating");}
                Thread.sleep(100); // milliseconds
                t.putdown(ph);  
            } catch (Exception e) {System.out.println("oops");}
        }
    }
}
class DiningPhilosophers {
    public static void main(String[] args) {
        Table table = new Table();
        for (int i = 0; i < 5; i++) new Philosopher(table, i).start();
        try {Thread.sleep(3000); // milliseconds to sleep before program stops
        } catch (Exception e) {}
    }
}

In [ ]:
!javac philosophers.java

In [ ]:
!java DiningPhilosophers

Analyze the fairness of both designs: is it possible that philosophers starve?

YOUR ANSWER HERE

1. With forks as monitors, a philosopher could starve, just as they can when using semaphores for forks.
2. With a table as a monitor, philosophers could also starve.

Discuss the merits of each of the two designs!

YOUR ANSWER HERE

- With forks as monitors, the advantage is that it allows two philosophers to pick up different forks at the same time, hence more parallelism is possible. Spurious wakeups do not occur, a notified philosopher can continue. However, philosophers have to be programmed to avoid deadlocks.

- With a table as a monitor, the advantage is that philosophers are oblivious to the way forks are allocated and deadlock is avoided. However, only one philosopher can interact with the table at a time, limiting parallelism. Spurious wakeups are possible in the table monitor, adding run-time overhead.